In [ ]:

train_data = '/content/drive/My Drive/rock paper scissor dataset/107582_309628_bundle_archive.zip'


In [ ]:
import zipfile

with zipfile.ZipFile(train_data, 'r') as z:
    z.extractall()

In [ ]:
import os
base_dir = '/content/train'
rock_dir = os.path.join(base_dir,'rock')
paper_dir = os.path.join(base_dir,'paper')
scissors_dir = os.path.join(base_dir,'scissors')

In [ ]:
print("Rock : ",len(os.listdir(rock_dir)))
print("Paper : ",len(os.listdir(paper_dir)))
print("Scissors : ",len(os.listdir(scissors_dir)))

Rock :  726
Paper :  712
Scissors :  750


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
batch_size=32
img_rows,img_cols=120,120
num_classes=3

datagen = ImageDataGenerator(
                rescale = 1/255,
                zoom_range = 0.20,
                rotation_range = 0.10,
                horizontal_flip = True,
                vertical_flip = True,
                fill_mode = 'nearest',
                validation_split = 0.4
)

train_generator = datagen.flow_from_directory(
        base_dir,
        target_size=(img_rows,img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='rgb',
        subset='training')
validation_generator = datagen.flow_from_directory(
        base_dir,
        target_size=(img_rows,img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='rgb',
        subset='validation')

Found 1314 images belonging to 3 classes.
Found 874 images belonging to 3 classes.


In [ ]:
print(validation_generator.class_indices)

{'paper': 0, 'rock': 1, 'scissors': 2}


In [ ]:
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:
# Create a Sequential model by passing a list of layers to the Sequential constructor
from tensorflow.keras.layers import Dropout
img_rows,img_cols=120,120
model = Sequential([Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu', 
                           input_shape=(img_rows,img_cols,3)),
                    MaxPooling2D(pool_size=(2,2)),
                    Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu'),
                    MaxPooling2D(pool_size=(2,2)),
                    Dropout(rate=0.25),
                    Conv2D(filters=64, kernel_size=(3,3), padding='same',activation='relu'),
                    MaxPooling2D(pool_size=(2,2)),
                    Conv2D(filters=128, kernel_size=(3,3), padding='same',activation='relu'),
                    MaxPooling2D(pool_size=(2,2)),
                    Dropout(rate=0.25),
                    Flatten(),
                    Dropout(rate=0.5),
                    Dense(units=512, activation='relu'),
                    Dense(units=num_classes, activation='softmax')])

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 120, 120, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 60, 60, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 60, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 64)        0

In [ ]:
import tensorflow as tf
accuracythreshold=96e-2

class AccCallback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('accuracy') >= accuracythreshold):   
          print("\nReached %2.2f%% accuracy, stop training!" %(accuracythreshold*100))   
          self.model.stop_training = True

In [ ]:
from tensorflow.keras.optimizers import RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [ ]:
nb_train_samples=1314
nb_validation_samples=874
epochs=55

history=model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    callbacks=[AccCallback()],
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/55
41/41 [==============================] - 10s 251ms/step - loss: 1.2234 - accuracy: 0.3775 - val_loss: 1.0217 - val_accuracy: 0.5116
Epoch 2/55
41/41 [==============================] - 10s 250ms/step - loss: 0.7712 - accuracy: 0.6521 - val_loss: 0.4009 - val_accuracy: 0.8843
Epoch 3/55
41/41 [==============================] - 10s 246ms/step - loss: 0.4301 - accuracy: 0.8346 - val_loss: 0.3128 - val_accuracy: 0.8854
Epoch 4/55
41/41 [==============================] - 10s 247ms/step - loss: 0.3320 - accuracy: 0.8838 - val_loss: 0.2858 - val_accuracy: 0.9074
Epoch 5/55
41/41 [==============================] - 10s 248ms/step - loss: 0.2783 - accuracy: 0.9017 - val_loss: 0.1964 - val_accuracy: 0.9410
Epoch 6/55
41/41 [==============================] - 10s 246ms/step - loss: 0.2464 - accuracy: 0.9165 - val_loss: 0.1813 - val_accuracy: 0.9491
Epoch 7/55
41/41 [==============================] - 10s 245ms/step - loss: 0.2241 - accuracy: 0.9150 - val_loss: 0.1661 - val_accuracy: 0.9387

In [ ]:
model.save('/content/drive/My Drive/rock paper scissor dataset/sps_model') 

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/My Drive/rock paper scissor dataset/sps_model/assets
